Packages Required:
    -pandas: 1.2.4
    -numpy: 1.18.0
    -transformers: 4.15.0
    -torch: 1.10.0
    -pickle: 1.6.0

In [ ]:
!pip install transformers

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import TransfoXLTokenizer, TransfoXLModel
import pickle

import warnings
warnings.filterwarnings("ignore")

In [34]:
#Read data from csv folder.
#The data used here is Shapes V4
data = pd.read_csv(r'D:\uOttawa\final_project\data\Book1.csv')

In [14]:
#Get the maximum length for a description
max_length = 0
for i in range(data.shape[0]):
    current_length = len(data.iloc[i,0].split())
    if(len(data.iloc[i,0].split()) > max_length):
        max_length = len(data.iloc[i,0].split())

In [9]:
#Tokenizer and transformer XL from hugging face
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
pretrained_model = TransfoXLModel.from_pretrained('transfo-xl-wt103')

Some weights of the model checkpoint at transfo-xl-wt103 were not used when initializing TransfoXLModel: ['crit.cluster_weight', 'crit.out_layers.0.weight', 'crit.out_projs.0', 'crit.out_projs.1', 'crit.out_layers.0.bias', 'crit.out_projs.2', 'crit.out_projs.3', 'crit.out_layers.2.bias', 'crit.out_layers.1.bias', 'crit.out_layers.1.weight', 'crit.out_layers.3.bias', 'crit.out_layers.2.weight', 'crit.cluster_bias', 'crit.out_layers.3.weight']
- This IS expected if you are initializing TransfoXLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TransfoXLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
#Dictionaries for attributes and their values to store them as numbers in pickle file
color_dict = {"none": 0, "mauve":1, "black":2, "blue":3, "green":4, "white":5, "red":6}
        
size_dict = {"none": 0, "small":1, "normal":2, "big":3}
        
shape_dict = {"none": 0,"ball":1, "cone":2, "cube":3, "cylinder":4, "torus":5}

action_dict = {"none": 0, "hover":1, "spin":2 ,"move":3, "stop": 4}

intent_dict = {"create":0, "animate":1, "modify":2}

In [12]:
#path to folder where pickle files will stored
output_folder = 'D:/uOttawa/final_project/pickle_inference/'

In [21]:
#Loop over all data in the csv file and get the number for the attribute based on the dictionary.
#Pass the description to the tokenizer to encode and then to the model to obtain the hidden_states and last_hidden then save them all in a pickle file
max_length = 16
for i in range(data.shape[0]):
    keys = {}
    keys["shape"] = shape_dict.get(data.iloc[i,1])
    keys["size"] = size_dict.get(data.iloc[i,2])
    keys["color"] = color_dict.get(data.iloc[i,3])
    keys["action"] = action_dict.get(data.iloc[i,4])
    keys["type"] = intent_dict.get(data.iloc[i,5])
    keys["description"] = data.iloc[i,0]
    #------------------------------------------------------------------------------
    encoded = tokenizer.encode(keys["description"], add_special_tokens=True)
    outputs = pretrained_model(torch.tensor(encoded).unsqueeze(0), output_hidden_states=True) 
    hidden_state = outputs.hidden_states[1] 
    last_hidden = outputs.last_hidden_state[:,-1,:]
    val = (max_length - list(hidden_state.shape)[1])

    hidden_state = nn.ConstantPad2d((0, 0, 0, val), 0)(hidden_state)

    keys['hidden_state'] = hidden_state
    keys['last_hidden'] = last_hidden

    desc_dict = {'desc_list': keys}
    save_file_name = output_folder + str(i)

    with open(save_file_name, 'wb') as outfile:
        pickle.dump(desc_dict,outfile)
     